In [ ]:
import os
import pandas as pd
from itertools import permutations
import numpy as np

# Assuming each project is represented by a separate file in the dataset directory
# Adjust the dataset_directory path as needed
dataset_directory = "/content/data"

def load_data(file_path):
    """
    Load the dataset from a specific file.
    Assuming the files are in CSV format. Adjust as needed.
    """
    df = pd.read_csv(file_path, sep=';')  # Specify semicolon as delimiter
    # Check if the last column is empty and drop it if it is
    if df.columns[-1] == '':
        df = df.iloc[:, :-1]  # Drop the last column

    print(df.columns) # Print column names to verify
    return df


def calc_bug_fix_data(project1, project2):
    """
    Calculate the bug fix data between two projects.
    This requires extracting the log data for various bug types and counting the number of bug fixes.
    """
    columns = [' bugs ', ' nonTrivialBugs ', ' majorBugs ', ' criticalBugs ', ' highPriorityBugs ']

    # Check if columns exist in both DataFrames
    for col in columns:
        if col not in project1.columns or col not in project2.columns:
            print(f"Warning: Column '{col}' not found in one or both DataFrames.")
            return np.nan # Return NaN if column is missing

    # Convert columns to numeric, handling potential errors
    for col in columns:
        project1[col] = pd.to_numeric(project1[col], errors='coerce')
        project2[col] = pd.to_numeric(project2[col], errors='coerce')

    log_data_project1 = project1[columns].sum().sum()
    log_data_project2 = project2[columns].sum().sum()

    return np.log(log_data_project1 + 1) - np.log(log_data_project2 + 1)

# List all project files in the dataset directory
project_files = [f for f in os.listdir(dataset_directory) if os.path.isfile(os.path.join(dataset_directory, f))]

# Container to hold the results
bug_fix_results = []

# Iterate over all permutations of project pairs
for project_file1, project_file2 in permutations(project_files, 2):
    project1 = load_data(os.path.join(dataset_directory, project_file1))
    project2 = load_data(os.path.join(dataset_directory, project_file2))

    bug_fix_data = calc_bug_fix_data(project1, project2)

    bug_fix_results.append({
        'project1': project_file1,
        'project2': project_file2,
        'bug_fix_data': bug_fix_data
    })

# Convert results to a DataFrame for better readability and further analysis
results_df = pd.DataFrame(bug_fix_results)
print(results_df)

# Optional: Save results to a CSV file
results_df.to_csv("bug_fix_data_results.csv", index=False)

Index(['classname ', ' numberOfVersionsUntil: ', ' numberOfFixesUntil: ',
       ' numberOfRefactoringsUntil: ', ' numberOfAuthorsUntil: ',
       ' linesAddedUntil: ', ' maxLinesAddedUntil: ', ' avgLinesAddedUntil: ',
       ' linesRemovedUntil: ', ' maxLinesRemovedUntil: ',
       ' avgLinesRemovedUntil: ', ' codeChurnUntil: ', ' maxCodeChurnUntil: ',
       ' avgCodeChurnUntil: ', ' ageWithRespectTo: ',
       ' weightedAgeWithRespectTo: ', ' bugs ', ' nonTrivialBugs ',
       ' majorBugs ', ' criticalBugs ', ' highPriorityBugs ', ' '],
      dtype='object')
Index(['classname ', ' numberOfVersionsUntil: ', ' numberOfFixesUntil: ',
       ' numberOfRefactoringsUntil: ', ' numberOfAuthorsUntil: ',
       ' linesAddedUntil: ', ' maxLinesAddedUntil: ', ' avgLinesAddedUntil: ',
       ' linesRemovedUntil: ', ' maxLinesRemovedUntil: ',
       ' avgLinesRemovedUntil: ', ' codeChurnUntil: ', ' maxCodeChurnUntil: ',
       ' avgCodeChurnUntil: ', ' ageWithRespectTo: ',
       ' weightedAgeWi

In [ ]:
import pandas as pd

# Load the existing bug fix data
# Replace 'bug_fix_data_results.csv' with the actual path to your CSV file containing the bug_fix_data
bug_data_df = pd.read_csv('/content/bug_fix_data_results.csv')

# Define the threshold for classification
threshold = 0

# Classify based on the threshold
# true = has bugs (positive bug_fix_data implies more bugs in project1 compared to project2)
# false = no bugs (non-positive bug_fix_data implies fewer or equal bugs in project1 compared to project2)
bug_data_df['actual_bug_existence'] = bug_data_df['bug_fix_data'] > threshold
bug_data_df['inferred_bug_existence'] = bug_data_df['bug_fix_data'] > threshold  # Using same classification logic

# Calculate TP, FN, FP, TN for each pair
def calculate_metrics(df):
    TP = ((df['actual_bug_existence'] == True) & (df['inferred_bug_existence'] == True)).sum()
    FN = ((df['actual_bug_existence'] == True) & (df['inferred_bug_existence'] == False)).sum()
    FP = ((df['actual_bug_existence'] == False) & (df['inferred_bug_existence'] == True)).sum()
    TN = ((df['actual_bug_existence'] == False) & (df['inferred_bug_existence'] == False)).sum()

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    return precision, recall

# Iterate over each pair of projects and calculate precision and recall
metrics_list = []
for index, row in bug_data_df.iterrows():
    project1 = row['project1']
    project2 = row['project2']

    # Filter the DataFrame for the current pair to calculate metrics
    filtered_df = bug_data_df[(bug_data_df['project1'] == project1) & (bug_data_df['project2'] == project2)]

    precision, recall = calculate_metrics(filtered_df)

    metrics_list.append({
        'project1': project1,
        'project2': project2,
        'precision': precision,
        'recall': recall
    })

# Convert the metrics list to a DataFrame for better readability and print results
metrics_df = pd.DataFrame(metrics_list)
print(metrics_df)

# Optional: Save the results to a CSV file
metrics_df.to_csv('precision_recall_results.csv', index=False)

       project1     project2  precision  recall
0       pde.csv    mylyn.csv        0.0     0.0
1       pde.csv   lucene.csv        1.0     1.0
2       pde.csv      jdt.csv        0.0     0.0
3       pde.csv  equinox.csv        1.0     1.0
4     mylyn.csv      pde.csv        1.0     1.0
5     mylyn.csv   lucene.csv        1.0     1.0
6     mylyn.csv      jdt.csv        1.0     1.0
7     mylyn.csv  equinox.csv        1.0     1.0
8    lucene.csv      pde.csv        0.0     0.0
9    lucene.csv    mylyn.csv        0.0     0.0
10   lucene.csv      jdt.csv        0.0     0.0
11   lucene.csv  equinox.csv        0.0     0.0
12      jdt.csv      pde.csv        1.0     1.0
13      jdt.csv    mylyn.csv        0.0     0.0
14      jdt.csv   lucene.csv        1.0     1.0
15      jdt.csv  equinox.csv        1.0     1.0
16  equinox.csv      pde.csv        0.0     0.0
17  equinox.csv    mylyn.csv        0.0     0.0
18  equinox.csv   lucene.csv        1.0     1.0
19  equinox.csv      jdt.csv        0.0 

In [ ]:
import pandas as pd
from itertools import permutations
import numpy as np
import os

# Define the dataset directory
dataset_directory = "/content/data"

# List all project files in the dataset directory
project_files = [f for f in os.listdir(dataset_directory) if os.path.isfile(os.path.join(dataset_directory, f))]

# Define the metrics columns
metrics_columns = [' bugs ', ' nonTrivialBugs ', ' majorBugs ', ' criticalBugs ', ' highPriorityBugs ']

def load_data(file_path):
    """
    Load the dataset from a specific file with semicolon delimiter.
    """
    return pd.read_csv(file_path, delimiter=';')

# Calculate mean metrics for each project
project_means = {}
for project_file in project_files:
    project_data = load_data(os.path.join(dataset_directory, project_file))
    project_means[project_file] = project_data[metrics_columns].mean()

# Apply compensation method
def apply_compensation(project_data, project_avg, other_avg):
    compensated_data = project_data.copy()
    for column in metrics_columns:
        compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
    return compensated_data

# Iterate over each pair of projects and calculate compensated bug fix data
metrics_list = []

for project_file1, project_file2 in permutations(project_files, 2):
    # Load project data
    project1_data = load_data(os.path.join(dataset_directory, project_file1)).copy()
    project2_data = load_data(os.path.join(dataset_directory, project_file2)).copy()

    # Apply compensation
    project1_avg = project_means[project_file1]
    project2_avg = project_means[project_file2]
    compensated_project1_data = apply_compensation(project1_data, project1_avg, project2_avg)

    # Compute bug_fix_data after compensation
    bug_fix_data = np.log(compensated_project1_data[metrics_columns].sum().sum() + 1) - np.log(project2_data[metrics_columns].sum().sum() + 1)

    metrics_list.append({
        'project1': project_file1,
        'project2': project_file2,
        'bug_fix_data': bug_fix_data
    })

# Convert metrics list to DataFrame
compensated_bug_data_df = pd.DataFrame(metrics_list)

# Define the classification threshold
threshold = 0

# Classify based on the threshold
compensated_bug_data_df['actual_bug_existence'] = compensated_bug_data_df['bug_fix_data'] > threshold
compensated_bug_data_df['inferred_bug_existence'] = compensated_bug_data_df['bug_fix_data'] > threshold  # Assuming same classification criteria

# Calculate TP, FN, FP, TN for each pair
def calculate_metrics(df):
    TP = ((df['actual_bug_existence'] == True) & (df['inferred_bug_existence'] == True)).sum()
    FN = ((df['actual_bug_existence'] == True) & (df['inferred_bug_existence'] == False)).sum()
    FP = ((df['actual_bug_existence'] == False) & (df['inferred_bug_existence'] == True)).sum()
    TN = ((df['actual_bug_existence'] == False) & (df['inferred_bug_existence'] == False)).sum()

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    return precision, recall

# Compute precision and recall for each pair
precision_recall_list = []

for project_file1, project_file2 in permutations(project_files, 2):
    # Filter DataFrame for the current pair
    filtered_df = compensated_bug_data_df[
        (compensated_bug_data_df['project1'] == project_file1) &
        (compensated_bug_data_df['project2'] == project_file2)
    ]

    if not filtered_df.empty:
        precision, recall = calculate_metrics(filtered_df)

        precision_recall_list.append({
            'project1': project_file1,
            'project2': project_file2,
            'precision': precision,
            'recall': recall
        })

# Convert the precision and recall metrics into a DataFrame and print results
precision_recall_df = pd.DataFrame(precision_recall_list)
print(precision_recall_df)

# Optional: Save the results to a CSV file with semicolon delimiter
precision_recall_df.to_csv('compensated_precision_recall_results.csv', index=False, sep=';')

<ipython-input-22-7b878493a59e>:31: RuntimeWarning: divide by zero encountered in scalar divide
  compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
<ipython-input-22-7b878493a59e>:31: RuntimeWarning: invalid value encountered in scalar divide
  compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
<ipython-input-22-7b878493a59e>:31: RuntimeWarning: divide by zero encountered in scalar divide
  compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
<ipython-input-22-7b878493a59e>:31: RuntimeWarning: invalid value encountered in scalar divide
  compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
<ipython-input-22-7b878493a59e>:31: RuntimeWarning: divide by zero encountered in scalar divide
  compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
<ipython-input-22-7b878493a59e>:

       project1     project2  precision  recall
0       pde.csv    mylyn.csv        0.0     0.0
1       pde.csv   lucene.csv        1.0     1.0
2       pde.csv      jdt.csv        1.0     1.0
3       pde.csv  equinox.csv        1.0     1.0
4     mylyn.csv      pde.csv        1.0     1.0
5     mylyn.csv   lucene.csv        1.0     1.0
6     mylyn.csv      jdt.csv        1.0     1.0
7     mylyn.csv  equinox.csv        1.0     1.0
8    lucene.csv      pde.csv        0.0     0.0
9    lucene.csv    mylyn.csv        0.0     0.0
10   lucene.csv      jdt.csv        0.0     0.0
11   lucene.csv  equinox.csv        1.0     1.0
12      jdt.csv      pde.csv        0.0     0.0
13      jdt.csv    mylyn.csv        0.0     0.0
14      jdt.csv   lucene.csv        1.0     1.0
15      jdt.csv  equinox.csv        1.0     1.0
16  equinox.csv      pde.csv        0.0     0.0
17  equinox.csv    mylyn.csv        0.0     0.0
18  equinox.csv   lucene.csv        0.0     0.0
19  equinox.csv      jdt.csv        0.0 

<ipython-input-22-7b878493a59e>:31: RuntimeWarning: invalid value encountered in scalar divide
  compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
<ipython-input-22-7b878493a59e>:31: RuntimeWarning: divide by zero encountered in scalar divide
  compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
<ipython-input-22-7b878493a59e>:31: RuntimeWarning: invalid value encountered in scalar divide
  compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
<ipython-input-22-7b878493a59e>:31: RuntimeWarning: divide by zero encountered in scalar divide
  compensated_data[column] = compensated_data[column] * (other_avg[column] / project_avg[column])
